# Primo progetto di Social Computing
**Anno accademico 2020-21**

**Allievi:**
* *Rizzetto Sasha [142660]*
* *Gasparollo Denis [143225]*
* *Bensi Martino [142793]*
* *Francescut Matteo [143352]*

## Importazione librerie

Le principali librerie impiegate sono: <br>
    •Tweepy: semplifica l’accesso e l’utilizzo delle API di Twitter <br>
    •NetworkX: permette la creazione, manipolazione e l’analisi di reti <br>
    •PyVis: gestisce rappresentazioni interattive di reti anche molto complesse

In [ ]:
from tweepy import TweepError as twE
import tweepy as tw
import os
import json
import random
from networkx.algorithms.approximation import clique as clq
import networkx as nx
from pyvis.network import Network
import pandas as pd

## Configurazione parametri API

In [ ]:
#Twitter API credentials
api_key = "FzUscQQNsdWhkxebVGFZyuHsh"
api_secret = "Tr1OqulWQIYYI6aEpWVOb7U5NL9DoRtCAviXh4tYg4LXg9sBZe"
access_token = "1040603971637923845-9uZqYh42gsCmwoDCB9nTcoSHrHwiV8"
access_secret = "OsZL0PZC9WDw6Xwcl2ASpIisyfkKW3Bwu4BomNgWxAcj0"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAJQUJQEAAAAA%2B5CTzK8KotMS%2FuB61Dlul7ijBE4%3D5uWZb7w1FPD6nb4SpvmnZ4hxGjqGwzfMquWPHQQooS3jkgNg67"

auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=240)

if api.verify_credentials:
    print ('Authentication completed successfully!')

## Definizione funzioni JSON

Abbiamo realizzato due funzioni utili per maneggiare i file json: <br>
    • **serialize_json**: serializza dei dati in formato JSON in un dato file <br>
    •  **read_json**: acquisisce dati in formato JSON da un dato dile <br>

In [ ]:
data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

## Definizione funzioni di download

Per gestire l'ottenimento dei dati necessari abbiamo sviluppato le seguenti due funzioni: <br><br>
    • **download**: Scarica i dati di un certo insieme di utenti (follower o following) e li salva in un file JSON <br>
                *accounts* --> vettore contenenti gli screen_name dei quali si vogliono scaricare i dati (vettore)<br>
                *what* --> specifica il cosa voglio scaricare (stringa) <br>
                *quantity* --> quantità di followers/followings si vogliono scaricare (intero) <br>
                *operation* --> funzione dell'API da chiamere (funzione) <br>

In [ ]:
def download(users, what, quantity, operation):
    i = 0

    for user in users:
        data= []
        print(f"Downloading {what} di {user}")

        for item in tw.Cursor(
            operation,
            screen_name=user,
            skip_status=True,
            include_user_entities=False
        ).items(quantity[i]):

            retriveData(item, data)

        serialize_json(data_folder, f"{user}-{what}s.json", data)
        print(f"ho serializzato un totale di {len(data)} dati \n")
        i += 1

• **retriveData**: estrae solo i dati necessari da quelli ricevuti dall'API <br>

In [ ]:
def retriveData(item, data):
    json_data = item._json
    found_follower = {}

    found_follower["id"] = json_data["id"]
    found_follower["name"] = json_data["name"]
    found_follower["screen_name"] = json_data["screen_name"]
    found_follower["location"] = json_data["location"]

    data.append(found_follower)

# 1) Download dei primi 5 utenti

Scaricamento utenti followers e following dei primi 5 account forniti:

* @mizzaro
* @damiano10
* @Miccighel_
* @eglu81
* @KevinRoitero

Nel frammento sottostante si possono notare 3 array:
* **accounts**: contiene i nomi degli utenti da scaricare
* **followers**: contiene il numero di follower degli utenti nell'array "accounts"
* **followings**: contiene il numero di following degli utenti nell'array "accounts"

In [ ]:
accounts = ["mizzaro" , "damiano10", "Miccighel_", "eglu81", "KevinRoitero"]
followers = [157, 786, 332, 539, 103]
followings = [331, 837, 212, 621, 256]

## Download follower

In [ ]:
download(accounts, "follower", followers, api.followers)
print("Download terminato")

## Download following

In [ ]:
download(accounts, "following", followings, api.friends)
print("Download terminato")

# 2) Download degli altri utenti

Per ciascuno degli utenti iniziali, sono stati scelti:
* 5 followers a caso, e per ognuno di questi sono stati scaricati 10 loro followers (*followers of followers*)
* 5 following a caso, e per ognuno di questi sono stati scaricati 10 loro following (*followings of followings*)

## Definizione funzioni utili

Per lo svolgimento di questo punto abbiamo deciso di creare le seguenti 3 funzioni di supporto:
* **rndIndex**: dati una quantità K e un valore massimo M ritorna un array contenente K indici compresi tra 0 e M

In [ ]:
def rndIndex(quantity, max):
    rndNum = []

    while len(rndNum) < quantity:
        pos = random.randint(0, max)

        if pos not in rndNum:
            rndNum.append(pos)
    return rndNum

* **retriveAccounts**: dato un array contenente gli id di alcuni utenti, e dato un file da cui leggere i dati, restituisce
un array con i relativi

In [ ]:
#recupera gli screen_name dei 5 account random
def retriveAccounts(rnd, file):
    data = read_json(f"{data_folder}/{file}")
    newAccounts = []

    for index in rnd:
        #controllo che aaccount selezionato abbia più di 10 followers
        item = data[index]
        newAccounts.append(item["screen_name"])

    return newAccounts

* **specificQuantityDownload**: analoga alla funzione *download()* vista prima, con la differenza che il parametro quantity
è un valore intero fisso e valido per tutte le iterazioni tra gli utenti (ad es. per ogni utente dobbiamo scaricare sempre
10 follower/following)

In [ ]:
def specificQuantityDownload(users, what, quantity, operation):

   for user in users:
        data= []
        print(f"Downloading {what} di {user}")

        for item in tw.Cursor(
            operation,
            screen_name=user,
            skip_status=True,
            include_user_entities=False
        ).items(quantity):

            retriveData(item, data)

        serialize_json(data_folder, f"{user}-{what}s.json", data)
        print(f"ho serializzato un totale di {len(data)} dati \n")

## Download followers of followers

In [ ]:
i = 0
for account in accounts:
    max = followers[i]

    rndNum = rndIndex(5, max)

    specificQuantityDownload(
        retriveAccounts(rndNum, f"{account}-followers.json"),
        f"{account}-followersOfFollower",
        10,
        api.followers
    )

    i += 1

print("Download terminato")

## Download followings of followings

In [ ]:
i = 0
for account in accounts:
    max = followings[i]

    rndNum = rndIndex(5, max)

    specificQuantityDownload(
        retriveAccounts(rndNum, f"{account}-followings.json"),
        f"{account}-followingsOfFollowing",
        10,
        api.friends
    )

    i += 1

print("Download terminato")

# 3) Download dei dettagli dei profili
Tramite la chiamata *api.get_user* andremo a scaricare i dettagli del profilo di tutti gli utenti
finora ottenuti.

Anche per questa parte andiamo a descrivere le funzioni di supporto sviluppate:

* **downloadUserDataFromString**: scarica i dettagli dell'utente, a partire dal suo screen_name.

In [ ]:
def downloadUserDataFromString(users, output):
    for user in users:
        print(f"Downloading userData di {user}")

        foundUser = api.get_user(user)
        retriveUserData(foundUser, output)

    return output

* **downloadUserDataFromObject**: scarica i dettagli dell'utente, a partire dal suo oggetto JSON.

In [ ]:
def downloadUserDataFromObject(users, output):
    for user in users:
        print(f"Downloading userData di {user['screen_name']}")
        try:
            foundUser = api.get_user(user["id"])
            retriveUserData(foundUser, output)
        except twE: #usato per gestire il caso in cui il profilo risulta impossibile da scaricare
            print("User not found!")

    return output

* **retriveUserData**: filtra soltanto i dati strettamente necessari.

In [ ]:
def retriveUserData(item, data):
    json_data = item._json

    profile_data = {}

    profile_data["id"] = json_data["id"]
    profile_data["name"] = json_data["name"]
    profile_data["screen_name"] = json_data["screen_name"]
    profile_data["location"] = json_data["location"]
    profile_data["description"] = json_data["description"]
    profile_data["followers_count"] = json_data["followers_count"]

    if profile_data not in data:
        data.append(profile_data)

## Download info utenti

In [ ]:
#lista dei file contenuti nella directory "data"
lstFile = os.listdir(data_folder)

if "usersData.json" not in lstFile:
    userData = []

    # 1. download user data degli utenti di partenza(mizzaro, damiano10,...)
    downloadUserDataFromString(accounts, userData)

    # 1. download user data degli altri utenti
    for file in lstFile:
       jsonItems = read_json(f"{data_folder}/{file}")

       downloadUserDataFromObject(jsonItems, userData)

    print(f"ho serializzato un totale di {len(userData)} dati \n")

    serialize_json(data_folder, f"usersData.json", userData)
else:
    print("I dati utente sono già stati serializzati in precedenza")

print("Download terminato")

# 4) Verifica delle relazioni
La seguente funzione va a verificare le relazioni (follow/following) tra i 5 utenti iniziali e gli altri
utenti scaricati, mostrando a terminale il risultato della verifica.

In [ ]:
def checkFriendships(nodes):
    for i in range(5):
        for j in range(10):
        # for j in (len(nodes)): #ABILITARE QUESTA PER VERIFICARE TUTTE LE COPPIE

            if i != j:
                friendship = api.show_friendship(source_id=nodes[i]["id"], target_id= nodes[j]["id"])

                if friendship[0].followed_by:
                    print(f"{nodes[i]['screen_name']} è seguito da {nodes[j]['screen_name']}")
                else:
                    print(f"{nodes[i]['screen_name']} non è seguito da {nodes[j]['screen_name']}")

                if friendship[0].following:
                    print(f"{nodes[i]['screen_name']} segue {nodes[j]['screen_name']}")
                else:
                    print(f"{nodes[i]['screen_name']} non segue {nodes[j]['screen_name']}")

                print("\n")

checkFriendships(read_json(f"{data_folder}/usersData.json"))
#esempio d'esecuzione di controllo dell'esistenza di relazioni
#(verifica per circa 50 coppie di coppie per utente iniziale)

# 5) Creazione della rete

In questa parte si va a creare il grafo tramite le funioni di NetworkX.

Abbiamo creato la seguente funzione **createIdDictionary** che restituisce un dizionario composto
da una coppia *screen_name* - *id*

In [ ]:
def createIdDictionary(lst):
    id = {}
    for item in lst:
        id.update({f"{item['screen_name']}": item["id"]})

    return id

Effettuiamo una verifica preliminare per stabilire se è già stato creato in precedenza (ossia
se il file è già presente), in tal caso non eseguiamo nuovamente la creazione.

In [ ]:
pwd = os.listdir()
lstFile = os.listdir(data_folder)

if "twtNet_graph.pkl" in pwd:
    twtNet = nx.read_gpickle("twtNet_graph.pkl")
    print("Recupero del grafo di twitter")
else:
    nodes = read_json(f"{data_folder}/usersData.json")

    #inizializzazione grafo
    twtNet = nx.DiGraph(group_members= ["Gasparollo Denis", "Francescut Matteo", "Bensi Martino", "Rizzetto Sasha"])

    #aggiunta nodi
    for node in nodes:
        twtNet.add_node(node["id"],
                        name= node["name"],
                        screen_name= node["screen_name"],
                        location= node["location"],
                        description= node["description"],
                        followers_count= node["followers_count"])
    print(nx.number_of_nodes(twtNet))

    #creazione dizionario screen_name - id
    idDictionary = createIdDictionary(nodes)

    #aggiunta archi
    for file in lstFile:

        if "usersData" not in file:
            splitedFileName = file.split(sep= "-", maxsplit=3)

            for user in read_json(f"{data_folder}/{file}"):

                if "followings" in splitedFileName[len(splitedFileName) - 1]:
                    twtNet.add_edge(idDictionary[splitedFileName[0]], user["id"])
                elif "followers" in splitedFileName[len(splitedFileName) - 1]:
                    twtNet.add_edge(user["id"], idDictionary[splitedFileName[0]])

    print("Il grafo è stato costruito con successo!")
    nx.write_gpickle(twtNet, "twtNet_graph.pkl")

# 6) Creazione visualizzazione interattiva

Definiamo ora i parametri per l'inizializzazione della visualizzazione

In [ ]:
net = Network(
    height = "100%",
    width = "100%",
    directed=True,
    bgcolor = "#222222",
    font_color = "white",
    heading = "Rete Sociale"
)

Adesso inizializziamo la mappa PyVis a partire dalla rete NetworkX

In [ ]:
net.barnes_hut()
net.from_nx(twtNet)
neighbor_map = net.get_adj_list()

for node in net.nodes:
    node["value"] = len(neighbor_map[node["id"]])

Mostriamo la mappa creata

In [ ]:
net.show("twtNet.html")

# 7) Verifiche di connessione e di bipartizione

Il risultato della verifica viene mostrato a terminale

In [ ]:
if "twtNet_graph.pkl" in os.listdir():
    twtNet = nx.read_gpickle("twtNet_graph.pkl")

    if  nx.is_connected(nx.to_undirected(twtNet)):
        print("Il grafo è connesso")
    else:
        print("Il grafo non è connesso")
    if nx.is_bipartite(twtNet):
        print("Il grafo è bipartito")
    else:
        print("Il grafo non è bipartito")
else:
    print("Grafo non ancora creato!")

Il grafo risulta **connesso** ma **non bipartito**.

# 8) Misurazioni di distanze

In questa sezione andiamo a effettuare le misurazioni di **centro, diametro e raggio**.

In [ ]:
if "twtNet_graph.pkl" in os.listdir():
    twtNet = nx.read_gpickle("twtNet_graph.pkl")
    undirTwtNet = nx.to_undirected(twtNet)

    center = nx.center(undirTwtNet)
    diameter = nx.diameter(undirTwtNet)
    radius = nx.radius(undirTwtNet)
else:
    print("Grafo non ancora creato!")

Il risultato della verifica viene mostrato a terminale

In [ ]:
print(f"Centro: {center}")
print(f"Diametro: {diameter}")
print(f"Raggio: {radius}")

# 9) Misurazioni di centralità

Vengono ora calcolate le seguenti misure di centralità:

* Betweenness centrality
* Closeness centrality
* Degree centrality
* In-degree centrality
* Out-degree centrality
* Page Rank

In [ ]:
if "twtNet_graph.pkl" in os.listdir():
    twtNet = nx.read_gpickle("twtNet_graph.pkl")

    betweennessCentrality = nx.betweenness_centrality(twtNet)
    closenessCentrality = nx.closeness_centrality(twtNet)
    degreeCentrality = nx.degree_centrality(twtNet)
    inCentrality = nx.in_degree_centrality(twtNet)
    outCentrality = nx.out_degree_centrality(twtNet)
    pageRank = nx.pagerank(twtNet)
    hits = nx.hits(twtNet, max_iter=150)

    print("Calcolo centralità terminato!")
else:
     print("Grafo non ancora creato!")

Il risultato del calcolo viene mostrato a terminale

In [ ]:
print(f"Betweenness centrality: {betweennessCentrality}")
print(f"Closeness centrality: {closenessCentrality}")
print(f"Degree centrality: {degreeCentrality}")
print(f"IN degree centrality: {inCentrality}")
print(f"OUT degree centrality: {outCentrality}")
print(f"Page rank: {pageRank}")
print(f"HITS hubs: {hits[0]}")
print(f"HITS authorities: {hits[1]}")

# 10) Creazione del sottografo

Creiamo il sottografo scegliendo come nodo centrale l'utente KevinRoitero (id: 3036907250).<br>

In [ ]:
pwd = os.listdir()

if "twtNet_graph.pkl" in pwd:
    twtNet = nx.read_gpickle("twtNet_graph.pkl")

    # partiamo dall'utente KevinRoitero (id: 3036907250)
    roiteroGraph = nx.ego_graph(twtNet, 3036907250, radius=1)
    # calcoliamo la cricca massima
    clique = clq.max_clique(nx.to_undirected(roiteroGraph))
    # creaiamo il sottografo della cricca massima
    cliqueGraph = twtNet.subgraph(clique)
    nx.write_gpickle(clique, "maxClique_graph.pkl")
else:
    print("Grafo non ancora creato!")

Stampiamo ora una visualizzazione del sottografo e la sua dimensione.

In [ ]:
nx.draw_networkx(cliqueGraph)
print(f"La dimensione del sottografo calcolato sulla cricca massima vale: {clq.large_clique_size(nx.to_undirected(roiteroGraph))}")

# 11) Calcolo della copertura minima degli archi

Il risultato verrà mostrato a terminale

In [ ]:
if "twtNet_graph.pkl" in os.listdir():
    twtNet = nx.read_gpickle("twtNet_graph.pkl")

    minCover = nx.min_edge_cover(nx.to_undirected(twtNet))
    print(f"Copertura minima arco: {minCover}")
else:
    print("Grafo non ancora creato!")

# 12) Osservazione del fenomeno "small-worldness"

In questa sezione effettueremo la stima di due coefficienti per osservare il fenomeno "small-worldness"
sulla rete oggetto di studio.<br>
Calcoleremo due valori:
* **Coefficiente omega**: compreso tra -1 e 1. Valori vicini allo 0 rappresentano bene la "small-worldness".
* **Coefficiente sigma**: Valori maggiori di 1 rappresentano bene la "small-worldness".


In [ ]:
if "twtNet_graph.pkl" in os.listdir():
    twtNet = nx.read_gpickle("twtNet_graph.pkl")

    #da vedere i parametri
    omega = nx.omega(nx.to_undirected(twtNet), niter=5, nrand=5)
    print(f"Coefficiente omega: {omega}")
    sigma = nx.sigma(nx.to_undirected(twtNet), niter=5, nrand=5)
    print(f"Coefficiente sigma: {sigma}")
else:
    print("Grafo non ancora creato!")

**Interpretazione**<br>
I risultati ottenuti ci inducono ad affermare, come ci si aspetterebbe,
che la rete possiede le caretteristiche di "small-worldness".
<br>Più nel dettaglio il *coefficiente omega* supporta pienamente questa ipotesi, essendo molto prossimo allo zero.
<br>Il *coefficiente sigma* invece, seppur prossimo all'uno, risulta abbastanza contraddittorio. Infatti il valore
atteso per una rete "small-world" dovrebbe essere superiore a 1.
<br>Ipotizziamo che lo scostamento verificato possa
essere imputabile alle semplificazioni apportate alla rete per motivi di limitazioni computazionali.

# 13) Calcolo della correlazione fra le misure di centralità

Riportiamo qui sotto il codice che restituisce in output una tabella rappresentativa di
alcune misure di centralità calcolate in precedenza.

In [ ]:
dfCentrality = pd.DataFrame(
    {"between_centrality": betweennessCentrality,
     "closeness_centrality": closenessCentrality,
     "degree_centrality": degreeCentrality,
     "in_degree_centrality": inCentrality,
     "out_degree_centrality": outCentrality,
     "page_rank": pageRank,
     "hits_hub": hits[0],
     "hirs_authorities": hits[1]
})

dfCentrality

## Coefficienti di correlazione di Pearson (Rho)

In [ ]:
dfCentrality.corr("pearson")

## Coefficienti di correlazione di Kendall (Tau)

In [ ]:
dfCentrality.corr("kendall")

# Appendice: codice aggiuntivo

## Rappresentazione grafica delle misure di centralità

La seguente funzione genera dei grafici delle varie misure di centralità e le salva in un file
*.pdf* all'interno della directory */centrality*

In [ ]:
import matplotlib.pyplot as plt
from pylab import rcParams

def draw(G, pos, measures, measureName):
    nodes = nx.draw_networkx_nodes(G, pos, node_size=5, cmap=plt.cm.plasma,
                                   node_color=list(measures.values()),
                                   nodelist=measures.keys(),
                                   linewidths=0.1,)

    edges = nx.draw_networkx_edges(G, pos)
    plt.title(measureName)
    plt.colorbar(nodes)
    plt.axis('off')
    rcParams['figure.figsize'] = 10, 10
    #plt.show()
    plt.savefig(f"centrality/{measureName}.pdf")

**Betweenness centrality**

In [ ]:
draw(twtNet,
     nx.spring_layout(twtNet),
     betweennessCentrality,
     "Between centrality")

**Closeness centrality**

In [ ]:
draw(twtNet,
     nx.spring_layout(twtNet),
     closenessCentrality,
     "Closeness centrality")

**Degree centrality**

In [ ]:
draw(twtNet,
     nx.spring_layout(twtNet),
     degreeCentrality,
     "Degree centrality")

**IN-Degree centrality**

In [ ]:
draw(twtNet,
     nx.spring_layout(twtNet),
     inCentrality,
     "IN-Degree centrality")

**OUT-Degree centrality**

In [ ]:
draw(twtNet,
     nx.spring_layout(twtNet),
     outCentrality,
     "OUT-Degree centrality")

**PageRank**

In [ ]:
draw(twtNet,
     nx.spring_layout(twtNet),
     pageRank,
     "PageRank")

**HITS (hubs)**

In [ ]:
draw(twtNet,
     nx.spring_layout(twtNet),
     hits[0],
     "HITS(hubs)")

**HITS (authorities)**

In [ ]:
draw(twtNet,
     nx.spring_layout(twtNet),
     hits[1],
     "HITS(authorities)")

## Download "globale" delle info utente

Abbiamo deciso, per motivi di memory usage, di recuperare solo porzione degli oggetti restituiti
dalle funzioni (api.followers, api.friends e api.get_user). <br>
Per completezza riportiamo qui sotto un esempio di download dell'intero oggetto: basta sostituire
 all'interno dei download la seguente funzione. <br>
L'esempio qui sotto è stato fatto per alcuni followers di mizzaro e per i dati utenti
(file: allmizzaro-followers.json)

**retriveAllData**

In [ ]:
def retriveAllData(item, data):
    json_data = item._json
    found_follower = json_data

    data.append(found_follower)

**download**

In [ ]:
def download(user, what, quantity, operation):
    data= []
    print(f"Downloading {what} di {user}")

    for item in tw.Cursor(
        operation,
        screen_name=user,
        skip_status=True,
        include_user_entities=False
    ).items(quantity):

        retriveAllData(item, data)

    serialize_json(os.getcwd(), f"all{user}-{what}s.json", data)
    print(f"ho serializzato un totale di {len(data)} dati \n")

**Esecuzione delle funzioni**

In [ ]:
#all followers info
download("mizzaro", "follower", 10, api.followers)

userInfo= []
user = "mizzaro"

print(f"Downloading info utente di {user}")

info = api.get_user(user)
retriveAllData(info, userInfo)
serialize_json(os.getcwd(), "mizzaro-complete-userInfo.json", userInfo)

## Merging file JSON

La funzione in questione effettua un'operazione di merge dei vari file JSON scaricati in un unico file.

In [ ]:
def mergeJSON():
    usersInfo = read_json(f"{data_folder}/usersData.json")
    lstFile = os.listdir(data_folder)

    #print(usersInfo[0]["followers"])

    for file in lstFile:
        splittedFileName = file.split(sep="-", maxsplit=3)

        for user in usersInfo:
            if (user["screen_name"] == splittedFileName[0]) and ("users" not in file):
                follows = read_json(f"{data_folder}/{file}")

                if "follower" in splittedFileName[len(splittedFileName) - 1]:
                    user["followers"] = follows
                elif "following" in splittedFileName[len(splittedFileName) - 1]:
                    user["followings"] = follows

                break

    serialize_json(os.getcwd(), "mergedData.json", usersInfo)

mergeJSON()